## 주가가 Crawling ㄱㄱ

In [ ]:
import FinanceDataReader as fdr
import pandas as pd

# 1. 데이터 불러오기 (삼성전자) 지정한 날짜 부터
df = fdr.DataReader('005930', '2025-05-01')

# 2. 필요한 컬럼만 추출 및 이름 변경
df = df.reset_index()
df = df[['Date', 'Open', 'Close']]
df.columns = ['date', 'open', 'close']

# 3. 날짜 포맷 변경 (YYYY.MM.DD)
df['date'] = df['date'].dt.strftime('%Y.%m.%d')

# 4. diff (종가 - 전날종가)
df['diff'] = df['close'].diff().fillna(0).astype(int)

# 5. rateOpen: (오늘시가 - 전날시가) / 전날시가 * 100  => 시가 등락률
df['rateOpen'] = df['open'].pct_change().fillna(0) * 100

# 6. rateClose: (오늘종가 - 전날종가) / 전날종가 * 100  => 종가 등락률
df['rateClose'] = df['close'].pct_change().fillna(0) * 100

# 7. rateDiff_OC: (오늘종가 - 오늘시가) / 오늘시가 * 100  => 일일 등락률
df['rateDiff_OC'] = ((df['close'] - df['open']) / df['open']).fillna(0) * 100

# 8. rateDiff_CO: (오늘시가 - 전날종가) / 전날종가 * 100  => 일간 등락률
df['rateDiff_CO'] = ((df['open'] - df['close'].shift(1)) / df['close'].shift(1)).fillna(0) * 100

# 9. 타입 정리
df['open'] = df['open'].astype(int)
df['close'] = df['close'].astype(int)
df = df[['date', 'open', 'close', 'diff', 'rateOpen', 'rateClose', 'rateDiff_OC', 'rateDiff_CO']]

# 10. 소수점 자리수 정리
df['rateOpen'] = df['rateOpen'].round(2)
df['rateClose'] = df['rateClose'].round(2)
df['rateDiff_OC'] = df['rateDiff_OC'].round(2)
df['rateDiff_CO'] = df['rateDiff_CO'].round(2)

print(df.head(10))


         date   open  close  diff  rateOpen  rateClose  rateDiff_OC  \
0  2025.05.02  55000  54300     0      0.00       0.00        -1.27   
1  2025.05.07  54500  54600   300     -0.91       0.55         0.18   
2  2025.05.08  55100  54600     0      1.10       0.00        -0.91   
3  2025.05.09  54700  54800   200     -0.73       0.37         0.18   
4  2025.05.12  55200  57600  2800      0.91       5.11         4.35   
5  2025.05.13  57700  56900  -700      4.53      -1.22        -1.39   
6  2025.05.14  57500  57400   500     -0.35       0.88        -0.17   
7  2025.05.15  57400  57300  -100     -0.17      -0.17        -0.17   
8  2025.05.16  57400  56800  -500      0.00      -0.87        -1.05   
9  2025.05.19  56400  55800 -1000     -1.74      -1.76        -1.06   

   rateDiff_CO  
0         0.00  
1         0.37  
2         0.92  
3         0.18  
4         0.73  
5         0.17  
6         1.05  
7         0.00  
8         0.17  
9        -0.70  


In [22]:
# DataFrame을 csv 파일로 출력력

uniqueTime = datetime.now().strftime('%Y%m%d_%H%M%S')
file_path_csv = f'data/{uniqueTime}_stockPrice.csv'
#file_path_excel = f'data/{uniqueTime}_stockPrice.xlsx'

df.to_csv(file_path_csv, index = False, encoding = "utf-8-sig")
print(f"{len(df)}일치 주가 데이터 수집하여 CSV 파일({file_path_csv})로 저장하였습니다.")

20일치 주가 데이터 수집하여 CSV 파일(data/20250602_144232_stockPrice.csv)로 저장하였습니다.


In [ ]:
# 다음 파이낸스 주가 정보


import requests
import pandas as pd

def get_daum_stock_data(stock_code):
    url = f'https://finance.daum.net/api/quote/A{stock_code}/days?symbolCode=A{stock_code}&page=1&perPage=30'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Referer': f'https://finance.daum.net/quotes/A{stock_code}'
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['data'])
        df = df[['date', 'tradePrice', 'changePrice', 'changeRate']]
        df.columns = ['date', 'close', 'diff', 'rate']

        # 날짜만 추출 (YYYY-MM-DD → YYYY.MM.DD)
        df['date'] = pd.to_datetime(df['date'].str[:10]).dt.strftime('%Y.%m.%d')

        # close, diff를 int로 변환 (소수점 버림)
        df['close'] = df['close'].astype(int)
        df['diff'] = df['diff'].astype(int)

        # rate를 % 수치로 변환 (예: 0.83 → 83.0)
        df['rate'] = (df['rate'] * 100).round(2)

        print(f"✅ API로 {len(df)}일치 데이터 수집 완료")
        return df
    else:
        print(f"❌ API 요청 실패: {response.status_code}")
        return None

# 사용 예시
df = get_daum_stock_data('005930')
if df is not None:
    print(df)


✅ API로 30일치 데이터 수집 완료
          date  close  diff  rate
0   2025.06.02  56900   700  1.25
1   2025.05.30  56200   100  0.18
2   2025.05.29  56100   200  0.36
3   2025.05.28  55900  2000  3.71
4   2025.05.27  53900  -800 -1.46
5   2025.05.26  54700   500  0.92
6   2025.05.23  54200  -500 -0.91
7   2025.05.22  54700 -1000 -1.80
8   2025.05.21  55700  -200 -0.36
9   2025.05.20  55900   100  0.18
10  2025.05.19  55800 -1000 -1.76
11  2025.05.16  56800  -500 -0.87
12  2025.05.15  57300  -100 -0.17
13  2025.05.14  57400   500  0.88
14  2025.05.13  56900  -700 -1.22
15  2025.05.12  57600  2800  5.11
16  2025.05.09  54800   200  0.37
17  2025.05.08  54600     0  0.00
18  2025.05.07  54600   300  0.55
19  2025.05.02  54300 -1200 -2.16
20  2025.04.30  55500  -300 -0.54
21  2025.04.29  55800     0  0.00
22  2025.04.28  55800   100  0.18
23  2025.04.25  55700     0  0.00
24  2025.04.24  55700     0  0.00
25  2025.04.23  55700   700  1.27
26  2025.04.22  55000  -400 -0.72
27  2025.04.21  55400   10

In [20]:
# DataFrame을 csv 파일로 출력력

uniqueTime = datetime.now().strftime('%Y%m%d_%H%M%S')
file_path_csv = f'data/{uniqueTime}_stockPrice.csv'
#file_path_excel = f'data/{uniqueTime}_stockPrice.xlsx'

df.to_csv(file_path_csv, index = False, encoding = "utf-8-sig")
print(f"{len(df)}일치 주가 데이터 수집하여 CSV 파일({file_path_csv})로 저장하였습니다.")

30일치 주가 데이터 수집하여 CSV 파일(data/20250602_112942_stockPrice.csv)로 저장하였습니다.
